# VOCS data structure 

Variables, Objectives, Constraints, and other Settings (VOCS) helps define our optimization problems. 

In [1]:
from xopt.vocs import VOCS

In [2]:
help(VOCS)

Help on class VOCS in module xopt.vocs:

class VOCS(xopt.pydantic.XoptBaseModel)
 |  VOCS(*, variables: Dict[str, types.ConstrainedListValue] = {}, constraints: Dict[str, types.ConstrainedListValue] = {}, objectives: Dict[str, xopt.vocs.ObjectiveEnum] = {}, constants: Dict[str, Any] = {}, observables: List[str] = []) -> None
 |  
 |  Variables, Objectives, Constraints, and other Settings (VOCS) data structure
 |  to describe optimization problems.
 |  
 |  Method resolution order:
 |      VOCS
 |      xopt.pydantic.XoptBaseModel
 |      pydantic.main.BaseModel
 |      pydantic.utils.Representation
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  as_yaml(self)
 |  
 |  constraint_data(self, data: Union[pandas.core.frame.DataFrame, List[Dict]], prefix: str = 'constraint_') -> pandas.core.frame.DataFrame
 |      Returns a dataframe containing constraint data transformed according to
 |      `vocs.constraints` such that values that satisfy each constraint are negative.
 |   

In [3]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)
vocs

VOCS(variables={'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}, constraints={'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]}, objectives={'c': 'MAXIMIZE', 'd': 'MINIMIZE'}, constants={'g': 1234}, observables=[])

In [4]:
# as dict
vocs.dict()

{'variables': {'a': [0.0, 1000.0], 'b': [-1.0, 1.0]},
 'constraints': {'e': ['LESS_THAN', 2.0], 'f': ['GREATER_THAN', 0.0]},
 'objectives': {'c': 'MAXIMIZE', 'd': 'MINIMIZE'},
 'constants': {'g': 1234},
 'observables': []}

In [5]:
#  re-parse dict
vocs2 = VOCS.parse_obj(vocs.dict())

In [6]:
# Check that these are the same
vocs2 == vocs

True

In [7]:
# This replaces the old vocs["variables"]
getattr(vocs, "variables")

{'a': [0.0, 1000.0], 'b': [-1.0, 1.0]}

In [8]:
vocs.objectives["c"] == 'MAXIMIZE'

True

In [9]:
# json
vocs.json()

'{"variables":{"a":[0.0,1000.0],"b":[-1.0,1.0]},"constraints":{"e":["LESS_THAN",2.0],"f":["GREATER_THAN",0.0]},"objectives":{"c":"MAXIMIZE","d":"MINIMIZE"},"constants":{"g":1234},"observables":[]}'

# Objective Evaluation

In [10]:
from xopt.vocs import form_objective_data, form_constraint_data, form_feasibility_data
import pandas as pd
import numpy as np

data = pd.DataFrame(vocs.random_inputs(10))
# Add some outputs
data["c"] = data["a"] + data["b"]
data["d"] = data["a"] - data["b"]
data["e"] = data["a"] * 2 + data["b"] * 2
data["f"] = data["a"] * 2 - data["b"] * 2
data.index = np.arange(len(data)) + 5  # custom index
data

a         b     g           c           d            e  \
5   772.372141  0.222951  1234  772.595092  772.149191  1545.190184   
6   639.612507  0.967282  1234  640.579790  638.645225  1281.159579   
7   365.660431  0.811871  1234  366.472302  364.848560   732.944604   
8   242.814691 -0.120613  1234  242.694078  242.935304   485.388157   
9   763.959468 -0.425893  1234  763.533575  764.385361  1527.067150   
10  709.376286 -0.630148  1234  708.746138  710.006433  1417.492276   
11  818.303426  0.447922  1234  818.751348  817.855503  1637.502696   
12  982.371596  0.835135  1234  983.206731  981.536461  1966.413462   
13  687.740359  0.131195  1234  687.871554  687.609164  1375.743107   
14  481.946759  0.740040  1234  482.686799  481.206719   965.373599   

              f  
5   1544.298381  
6   1277.290450  
7    729.697121  
8    485.870609  
9   1528.770723  
10  1420.012867  
11  1635.711007  
12  1963.072921  
13  1375.218329  
14   962.413438

In [11]:
vocs.objectives

{'c': 'MAXIMIZE', 'd': 'MINIMIZE'}

In [12]:
# These are in standard form for minimization
form_objective_data(vocs.objectives, data)

objective_c  objective_d
5   -772.595092   772.149191
6   -640.579790   638.645225
7   -366.472302   364.848560
8   -242.694078   242.935304
9   -763.533575   764.385361
10  -708.746138   710.006433
11  -818.751348   817.855503
12  -983.206731   981.536461
13  -687.871554   687.609164
14  -482.686799   481.206719

In [13]:
# This is also available as a method
vocs.objective_data(data)

objective_c  objective_d
5   -772.595092   772.149191
6   -640.579790   638.645225
7   -366.472302   364.848560
8   -242.694078   242.935304
9   -763.533575   764.385361
10  -708.746138   710.006433
11  -818.751348   817.855503
12  -983.206731   981.536461
13  -687.871554   687.609164
14  -482.686799   481.206719

In [14]:
# use the to_numpy() method to convert for low level use.
vocs.objective_data(data).to_numpy()

array([[-772.59509175,  772.14919055],
       [-640.57978969,  638.64522502],
       [-366.47230199,  364.84856036],
       [-242.69407826,  242.93530432],
       [-763.53357493,  764.38536135],
       [-708.74613799,  710.00643342],
       [-818.75134786,  817.85550332],
       [-983.20673117,  981.5364605 ],
       [-687.87155354,  687.60916429],
       [-482.68679927,  481.20671891]])

In [15]:
vocs.constraint_data(data)

constraint_e  constraint_f
5    1543.190184  -1544.298381
6    1279.159579  -1277.290450
7     730.944604   -729.697121
8     483.388157   -485.870609
9    1525.067150  -1528.770723
10   1415.492276  -1420.012867
11   1635.502696  -1635.711007
12   1964.413462  -1963.072921
13   1373.743107  -1375.218329
14    963.373599   -962.413438

In [16]:
vocs.feasibility_data(data)

feasible_e  feasible_f  feasible
5        False        True     False
6        False        True     False
7        False        True     False
8        False        True     False
9        False        True     False
10       False        True     False
11       False        True     False
12       False        True     False
13       False        True     False
14       False        True     False

# Error handling

In [17]:
Y = """
variables:
  a: [0, 1e3] # Note that 1e3 usually parses as a str with YAML. 
  b: [-1, 1]
objectives:
  c: maximize
  d: minimize 
constraints:
  e: ['Less_than', 2]
  f: ['greater_than', 0]
constants:
  g: 1234

"""

vocs = VOCS.from_yaml(Y)

In [18]:
d = {'a': [1,2,3]}

df = pd.DataFrame(d)
df2 = pd.DataFrame(df).copy()

df2['b'] = np.nan
df2['b'] - 1

0   NaN
1   NaN
2   NaN
Name: b, dtype: float64

In [19]:
data['a']  = np.nan

In [20]:
a = 2
def f(x=a):
    return x
a=99
f()

2

In [21]:
pd.DataFrame(6e66, index=[1,2,3], columns=['A'])

A
1  6.000000e+66
2  6.000000e+66
3  6.000000e+66

In [22]:
# These are in standard form for minimization

data = pd.DataFrame({'c':[1,2,3,4]}, index=[9,3,4,5])

form_objective_data(vocs.objectives, data)

objective_c  objective_d
9         -1.0          inf
3         -2.0          inf
4         -3.0          inf
5         -4.0          inf